In [156]:
import glob
import codecs
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('type it in the main room or a pm to the')
blocklist.append('The topic is')
blocklist.append('Topic set by')

all_separated = []
all_posts = []
all_posts_410 = []
all_posts_501 = []
authors = {}
authors_410 = {}
authors_501 = {}

def parse_all(files, separate):
    for file in glob.glob(files):
        lines = codecs.open(file, mode='r', encoding='utf-8')

        for line in lines:
            if not in_blocklist(blocklist, line.strip()): # blocked?
                if line[0] is '[' and line.find("<") is not -1:
                    s = line
                    sentence = s[s.find(">")+2:]
                    time = s[s.find("[")+1:s.find("]")]
                    author = s[s.find("<")+1:s.find(">")]
                    
                    p = (time+' ' + author, sentence.strip().lower())
                    all_posts.append(p)
                    if separate in file:
                        all_separated.append(p)

                    if author not in authors: # save things
                        authors[author] = set() # init list for author
                    authors[author].add(sentence.strip().lower()) 
                else:
                    if line.find('*') is not -1 and ") has joined #" not in line and ") Quit (" not in line and ") has left #" not in line and "6A14thena14, 06y14o14u 06n14eve14r" not in line and " sets mode: +" not in line and " is now known as " not in line:
                        time = line[line.find("[")+1:line.find("]")]
                        author_beginning = line.find("* ")
                        aa = line[author_beginning+2:]
                        aa_end = author_beginning + 2 + aa.find(" ")
                        author = line[author_beginning+2:aa_end]
                        sentence = line[aa_end + 1:]
                        
                        p = (time+' ' + author, sentence.strip().lower())
                        all_posts.append(p)
                        if separate in file:
                            all_separated.append(p)

                        if author not in authors: # save things
                            authors[author] = set() # init list for author
                        authors[author].add(sentence.strip().lower()) 
                    if ") has joined #" in line or ") Quit (" in line or ") has left #" in line:
                        time = line[line.find("[")+1:line.find("]")]
                        author_beginning = line.find("* ")
                        aa = line[author_beginning+2:]
                        aa_end = author_beginning + 2 + aa.find(" ")
                        author = line[author_beginning+2:aa_end]
                        if ") has joined #" in line:
                            author += " JOINED THE ROOM"
                        else:
                            author += " LEFT THE ROOM"
                        sentence = ""
                        p = (time+' ' + author, sentence)
                        all_posts.append(p)
                        if separate in file:
                            all_separated.append(p)
                            
                        if author not in authors: # save things
                            authors[author] = set() # init list for author
                        authors[author].add(sentence) 

    for line in codecs.open('1006/410_utf8.txt', mode='r', encoding='utf-8'):
        if ':' in line and not in_blocklist(blocklist, line.strip()): # blocked?:
            sentence = line[line.find(":"):]
            all_posts.append((author, sentence.strip().lower()))
            all_posts_410.append((author, sentence.strip().lower()))

            author = line[:line.find(":")]
            if author not in authors: # save things
                authors[author] = set() # init list for author
            if author not in authors_410:
                authors_410[author] = set()
            authors[author].add(sentence.strip().lower())
            authors_410[author].add(sentence.strip().lower())


    for s in codecs.open("1006/501_utf8.txt", mode='r', encoding='utf-8').readlines():
        if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
            if not in_blocklist(blocklist, s.strip()): # blocked?
                sentence = s[s.find(">")+2:]
                all_posts.append((author, sentence.strip().lower()))
                all_posts_501.append((author, sentence.strip().lower()))

                author = s[s.find("<")+1:s.find(">")]
                if author not in authors: # save things
                    authors[author] = set() # init list for author
                if author not in authors_501:
                    authors_501[author] = set()
                authors[author].add(sentence.strip().lower())
                authors_501[author].add(sentence.strip().lower()) 
        
parse_all("xxxchatters_logs/*.log", "#chat")
#for file in glob.glob("xxxchatters_logs/*.log"):
print(len(all_posts))
print(len(all_separated))
print(len(authors))
print(len(authors_410))
print(len(authors_501))
print(len(all_posts_410))
print(len(all_posts_501))

479404
158959
29086
40
73
304
238


In [ ]:
# testing to bring in notices
lines = codecs.open("1206/xxxchatters_original_0223_#chat.txt", mode='r', encoding='utf-8')

for line in lines:
    if not in_blocklist(blocklist, line.strip()): # blocked?
        if line[0] is '[' and line.find("<") is not -1:
            s = line
            sentence = s[s.find(">")+2:]
            time = s[s.find("[")+1:s.find("]")]
            author = s[s.find("<")+1:s.find(">")]

            p = (time+' ' + author, sentence.strip().lower())
            #all_posts.append(p)
            #if separate in file:
            #    all_separated.append(p)

            #if author not in authors: # save things
            #    authors[author] = set() # init list for author
            #authors[author].add(sentence.strip().lower()) 
        else:
            if line.find('*') is not -1 and ") has joined #" not in line and ") Quit (" not in line and ") has left #" not in line and "6A14thena14, 06y14o14u 06n14eve14r" not in line and " sets mode: +" not in line and " is now known as " not in line:
                time = line[line.find("[")+1:line.find("]")]
                author_beginning = line.find("* ")
                aa = line[author_beginning+2:]
                aa_end = author_beginning + 2 + aa.find(" ")
                author = line[author_beginning+2:aa_end]
                sentence = line[aa_end + 1:]
                print(author)
                print(time)
                print(sentence)
                print(line)
            #print(line)

In [ ]:
for a in authors['carrol']:
    print(a)

In [157]:
import operator
import string
word_counts = dict()
translator = str.maketrans('', '', string.punctuation)

def preprocess_word(word):
    word = word.replace('\x02', '')
    word = word.replace('\x03', '')
    word = word.replace('.', '')
    word = ''.join([i for i in word if not i.isdigit()])

    #word = word.translate(translator)
    word = ''.join(e for e in word if e.isalnum())
    return word


for post in all_posts:
    word_list = post[1].split()
    for word in word_list:
        word = preprocess_word(word)
        #word = word.replace('\x03', '')
        #word = word.replace('.', '')

        #word = word.translate(translator)
        #word = ''.join(e for e in word if e.isalnum())
        if len(word) < 1:
            continue
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

sorted_word_counts = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

def get_sorted_word_counts():
    return sorted_word_counts

v = get_sorted_word_counts()
print(len(v))

49231


In [158]:
def get_topn_percent(perc):
    counts = get_sorted_word_counts()
    l = len(counts)
    max_index = int(perc*l)
    top_n = []
    #print(counts[:max_index])
    for t in counts[:max_index]:
        top_n.append(t[0])
    return top_n

def get_all_by_max_appearance(app):
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] >= app:
            processed_counts.append(c[0])
        
    return processed_counts

def get_by_appearance_list(appearance):
    # appearance = [2, 3, 4] # number indicating times of appearance
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] in appearance:
            processed_counts.append(c[0])
        
    return processed_counts
    
#topn = get_topn_percent(0.001)

selected_appearances = get_by_appearance_list([110, 112])
print(selected_appearances)

#maxapp = get_all_by_max_appearance(2)
#file = open("1106/chat_room_word_appearance.txt", 'w')
#for e in maxapp:
#    try:
#        file.write(e[0] + ", " + str(e[1]) + '\n')
#    except:
#        pass
#file.close()

['human', 'quickly', 'wine', 'irc', 'extra', 'large', 'arm', 'past', 'rl', 'size', 'summer', 'leaves', 'devil', 'jag', 'able', 'naderaax', 'dylan', 'firm', 'kinky']


In [136]:
def remove_non_frequent(allowed_words, post, replace=True):
    good_words = []
    for word in post.lower().split():
        pword = preprocess_word(word)
        if pword in allowed_words:
            good_words.append(pword)
        else:
            if replace:
                replacement = ""
                for char in word:
                    replacement += "-"
                good_words.append(replacement)
    
    return " ".join(good_words)


def remove_by_probability(allowed_words, always_allowed, post, chance, replace=True):
    import random
    good_words = []
    for word in post.lower().split():
        #pword = preprocess_word(word)
        if word in always_allowed:
            good_words.append(word)
            continue
            
        if word in allowed_words:
            good_words.append(word)
        else:
            if replace:
                rv = random.uniform(0.0, 1.0)
                if rv < chance:
                    replacement = ""
                    for char in word:
                        replacement += "-"
                    good_words.append(replacement)
                else:
                    good_words.append(word)
    
    return " ".join(good_words)

sentence_to_test = "Jus to keep being evil<<<< wops Athena on the backside with a new nerf bat"
print(remove_non_frequent(get_topn_percent(0.1), sentence_to_test))

print(remove_by_probability(get_topn_percent(0.1), get_topn_percent(0.01), sentence_to_test, 0.5))

--- to keep being evil ---- athena on the -------- with a new nerf bat
--- to keep being evil<<<< ---- athena on the -------- with a new nerf bat


In [214]:
sentence_to_test = "bops saffron on the backside with one of my new nerf bats"

v = 0.0
while v <= 1.0:
    v += 0.04
    print(remove_non_frequent(get_topn_percent(v), sentence_to_test))
    

---- saffron on the -------- with one of my new ---- ----
---- saffron on the -------- with one of my new nerf ----
bops saffron on the -------- with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron o

In [234]:
import numpy as np

def get_by_probability(authors_list, count, topn = 30):
    author_probability = {}
    
    for authors in authors_list:
        for key, value in authors.items():
            if key not in author_probability:
                author_probability[key] = len(value)
            else:
                author_probability[key] += len(value)
                
    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    
    #print(author_probability_counts[:topn])

    sum_sentences = 0
    for v in author_probability_counts[:topn]:
        sum_sentences += v[1]

    author_probability_final = {}
    author_list = []
    prob_list = []

    for v in author_probability_counts[:topn]:
        author_probability_final[v[0]] = v[1]/float(sum_sentences)
        author_list.append(v[0])
        prob_list.append(v[1]/float(sum_sentences))

    final_autor_list = np.random.choice(
      author_list, 
      count,
      p=prob_list
    )

    return final_autor_list

def get_topn_authors(author_lists, n):
    author_probability = {}
    
    for author_list in author_lists:
        for key, value in author_list.items():
            if key not in author_probability:
                author_probability[key] = len(value)
            else:
                author_probability[key] += len(value)

    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    
    return author_probability_counts[:n]

#print(get_by_probability(authors, 20))
#print(get_by_probability(authors_410, 20))
#print(get_by_probability(authors_501, 20))

print(get_topn_authors([authors_410,authors_501], 30))
#print(get_by_probability([authors_410,authors_501], 200))

[('@erotic_kitty', 65), ('carrol', 53), ('+DJ`Mercury', 36), ('Louise', 28), ('@erotic_kitty{S}', 27), ('Valkyrie', 21), ('Threeleggedcat', 20), ('PlayfulBBC', 18), ('@saffron{WH}', 17), ('WhoGiveSaDamn', 15), ('Silver_haired`Fox', 14), ('sweet_teresa', 10), ('Jothom', 8), ('Woman', 8), ('guynextdoor', 7), ('Sanger', 7), ('Anastatia', 7), ('^fran', 7), ('Cruel`Intentions', 7), ('Jaems', 6), ('+DJ`South', 6), ('JFetish', 5), ('FunkyBoogieKing', 5), ('@DJ`liltech', 4), ('MeanMark1', 4), ('Handyman', 4), ('gracie', 4), ('saffron{WH}', 3), ('FriendlyMonster', 3), ('rst37', 3)]


In [16]:
# load osc config and parse for later use
import csv

def load_osc_config(file_name):
    config = {}
    with open(file_name, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter='\t', quotechar='|')
        index = 0
        for row in reader:
            if index > 0:
                #print(row)
                for i in range(len(row)):
                    config[i].append(row[i])
            else:
                config[0] = []
                config[1] = []
                config[2] = []
                config[3] = []
                config[4] = []
            index += 1
    
    return config
            
osc_config = load_osc_config('osc_config.csv')

In [212]:
import random
import sys

def generate_lines_linear(count):
    return_lines = []
    while len(return_lines) < count:
#    for i in range(1, count):
        sentence = random.choice(all_posts)[1]
        perc = len(return_lines)/float(count)

        topn = get_topn_percent(perc)
        #print(perc)
        #print(len(topn))
        #print(sentence)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        if len(stripped_post) > 20 and stripped_post not in return_lines:
            return_lines.append(stripped_post)
        #print(str(perc) + ": " + stripped_post)
        
    return return_lines
        
def generate_lines_from(fromp, step, count):
    for i in range(count):
        sentence = random.choice(all_posts)
        perc = fromp + i * step

        topn = get_topn_percent(perc)
        #print(len(topn))
        stripped_post = remove_non_frequent(topn, sentence[1].strip().lower())
        print("{:.6f}".format(perc) + ": " + stripped_post)
        
def generate_by_osc(config, lists, offset=90):
    assert(len(config) is not len(lists), "len of config and lists needs to be the same")

    index = 0
    possible_words = []
    # length of generated sentences is determined by osc/curve data
    length = len(config[0])
    for l in lists:
        _config = config[index]
        _min = l[0]
        _max = l[1]
        app_list = []
        for i in range(_min, _max+1):
            app_list.append(i)
        
        # all allowed words
        selected_appearances = get_by_appearance_list(app_list)
        possible_words.append((selected_appearances, _config))
        
        index += 1
    
    # all words with min appearance of 500 are always allowed
    always_allowed = get_all_by_max_appearance(500)
        
    finished_sentences = []
    for i in range(length):
        #s = random.choice(all_posts)[1]
        s = all_separated[offset+i][1]
        a = all_separated[offset+i][0]
        # don't process djs
        if 'DJ`Mercury' in a or 'DJ`Protea' in a:
            finished_sentences.append((a, s))
            continue
            
        for p in possible_words:
            _words = p[0]
            probability = float(p[1][i])
            s = remove_by_probability(_words, always_allowed, s, probability)
        
        finished_sentences.append((a, s))
    
    return finished_sentences

def render_plain(out_filename, line_count, offset):
    outfile = codecs.open(out_filename, mode='w', encoding='utf-8')
    for i in range(line_count):
        s = all_separated[offset+i][1]
        a = all_separated[offset+i][0]
        outfile.write(a + ': ' + s + '\n')
    outfile.close()

def render_by_osc(osc_config, out_filename, offset):
    lists = []
    lists.append((1, 5))
    lists.append((6, 20))
    lists.append((21, 50))
    #lists.append((51, 100))
    #lists.append((101, 500))
    osc_config = load_osc_config(osc_config)
    generated = generate_by_osc(osc_config, lists, offset)

    outfile = codecs.open(out_filename, mode='w', encoding='utf-8')
    for line in generated:
        outfile.write(line[0] + ': ' + line[1] + '\n')
    outfile.close()

    

<ipython-input-212-871762c2362e>:33: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(config) is not len(lists), "len of config and lists needs to be the same")


In [ ]:
auth = get_by_probability([authors_410,authors_501], 300)
print(auth)

In [ ]:
#random.seed(1)

def generate_lines_maxapp(count, maxapp):
    return_lines = []
    while len(return_lines) < count:
#    for i in range(1, count):
        #perc = len(return_lines)/float(count)
        #perc = 0.5
        sentence = random.choice(all_posts)[1]
        if 'radio meltdown' in sentence:
            continue
        #topn = get_all_by_max_appearance(maxapp)
        topn = get_topn_percent(0.025)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        if len(stripped_post) > 80 and stripped_post not in return_lines:
            return_lines.append(stripped_post)
        #print(str(perc) + ": " + stripped_post)
        
    return return_lines

outfile = codecs.open('1306_only_top0.025_longerthan80_01.txt', mode='w')

# 20 top authors from 401 & 510
auth_text = get_by_probability([authors_410,authors_501], 300, 20)
auth_io = get_topn_authors([authors_410,authors_501], 30)[-10:] # get_by_probability([authors_410,authors_501], 50, 30)[-10:]
#print(auth_text)
#print(auth_io)

# 300 lines
index = 0
lines = generate_lines_maxapp(50, 200)
for line in lines:
    l = ''.join(line)
    outfile.write(auth[index] + ': ' + l + '\n')
    print(auth_text[index] + ': ' + l)
    if random.uniform(0, 1) > 0.7:
        lj = ""
        if random.uniform(0, 1) > 0.5:
            lj = " LEFT THE ROOM"
        else:
            lj = " JOINED THE ROOM"
        print(random.choice(auth_io)[0] + lj)
        outfile.write(random.choice(auth_io)[0] + lj + '\n')
    index += 1

outfile.close()
# random in/outs for last 10
# top_n 2000 / 2500



In [351]:
# attempt to find similar sentences by matching vocabulary 

#li = ['her', 'her', 'her', 'he']
li = ['are', 'fake']
li = ['what', 'she', 'does']
#li = ['what', 'what', 'what']
li = ['is', 'is', 'is', 'is']
li = ['being'] * 3

#li = ['good', 'good', 'am']
#li = ['here', 'here', 'here']
#li = ['nice', 'nice', 'very']
#li = ['well', 'well']
#li = ['ll', 'll', 'll', 'll']
li = ['no'] * 4
#li = ['just', 'well', 'i']
index = 0
for post in all_posts:
    p = post[1]
    #print(p)
    success = True
    for l in li:
        if l in p:
            index = p.find(l)
            length = len(l)
            endindex = index + length
            subs = p[index:endindex] 
            leftover = p[:index] + p[endindex:]
            p = leftover
        else:
            success = False
            #print(post[1] + ' failed')
    if success and len(post[1]) < 80: 
        #print(len(post[1]))
        print(post[1])
    index += 1
        


no no no no
annnnoooooooorrrrrraaaaaaaaaaaaa
i know you knew, i just wanna know the interest of it, knowing that you know
we have snow nordicgoddess but not enough to be snowed in .... yet lol
no pwincess_aria no no no no
pretends to know nothing then no one expects me to know anything
no nononononono
nnnnnoooo more poucing, hehe
nnnnnooooooooooooooooooooooo
yeah i know dj`annora not a good thing right now
07,01 limits: 00 no pee and scat, no permanent injuries, no animals nor kids
lol no no no no
nnnnnnnnnnnooooooooooooo


In [ ]:
# todo: 1406
# insert sentence into parser
# extract gramatical structure
# try to match through corpus
# use pre-processed text, with dashes 


In [204]:
# CURVES generation

random.seed(1)
#render_plain('1306_removed_digits_entrances_01_ORIGINAL.txt', 500, 3860)
#render_by_osc('osc_config_500_late_slowexp.csv', '1306_cleaner_group1&2&3_01.txt', 3860)
print('finito')
#generate_lines_linear(20)

generate_lines_from(0.00, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.10, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.20, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.50, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.80, 0.0001, 10)

finito
0.000000: 
0.000100: 
0.000200: 
0.000300: ------- --- ---- --- --
0.000400: 
0.000500: 
0.000600: 
0.000700: hey -----
0.000800: 
0.000900: 
####################################################################################
0.100000: 
0.100100: 
0.100200: 
0.100300: 
0.100400: 
0.100500: 
0.100600: ill pass
0.100700: 
0.100800: hi matt
0.100900: 
####################################################################################
0.200000: ---------
0.200100: --- radio meltdown djwildkat is playing -- rm dj promo - djcaveman
0.200200: you two are disgusting
0.200300: hugs biffyyy
0.200400: 
0.200500: 
0.200600: hi ------------
0.200700: radiomeltdown radio meltdown djbooya is off --- coming up radio meltdown djmercury
0.200800: --- radio meltdown djwof is playing -- randy travis - three wooden crosses
0.200900: oh hell
####################################################################################
0.500000: 
0.500100: --- radio meltdown djmercury is playing -- david bowi

In [101]:
# saving word groups
lists = []
#lists.append((1, 5))
#lists.append((6, 20))
#lists.append((21, 50))
#lists.append((51, 100))
#lists.append((101, 500))
lists.append((3000, 10000))

outfile = codecs.open('3000_10000_groups.txt', 'w')

for l in lists:
    possible_words = []
    #_config = config[index]
    _min = l[0]
    _max = l[1]
    app_list = []
    for i in range(_min, _max+1):
        app_list.append(i)

    # all allowed words
    selected_appearances = get_by_appearance_list(app_list)
    outfile.write(' '.join(selected_appearances))
    #possible_words.append((selected_appearances, _config))
    
outfile.close()
    


In [ ]:
# postprocess act1&2

for sentence in all_posts_501:
    print(sentence[0] + " -> " + remove_non_frequent(get_topn_percent(0.01), sentence[1]))